# Birefringence calibration -- Heavy Duty tape

This script implements the method of [\[Belendez et al. 2009\]](https://physlab.lums.edu.pk/images/3/30/Cellophane2.pdf) to acquire the birefringence value of a birefringent material (tape) based on a series of measurements under different single-wavelength illuminants.

Units:  
angles: degrees  
wavelengths: nanometers  
thickness: nanometers  

Tape:  
Heavy Duty: 3.1 mil / 78740 nm thick  

Laser wavelengths:  
650 nm (red)  
532 nm (green)  
405 nm (purple)

In [1]:
%cd ".."

import os
import itertools

import rawpy
import numpy as np
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm.notebook import trange, tqdm

import sensor_data
import color_utils
import tape_data
from experiment import *

/Users/kate/hyperspectral


In [2]:
# Laser params
laser_wavelengths = np.array([405.0, 532.0, 650.0])

# Tape params
thickness = tape_data.THICKNESS_HD # Heavy Duty

# Measurement analyzer (polarizer) angles
theta = np.linspace(0.0, 180.0, 37)

# Percentile to use for intensity for image processing
percentile = 90.0

# Crop to reasonable square around laser glow
crop_y0 = 750
crop_y1 = 2750
crop_x0 = 1500
crop_x1 = 3500

In [3]:
# Load image files
file_prefix = "data/real_world/03-30/"

# Assumes theta measurements (per wavelength) are in order 
# starting with "base" number (where theta = 0)
img_start = np.array([117, 3, 946])

# Intensity measurements
measurements = np.zeros((len(laser_wavelengths),len(theta)))

for i in trange(len(theta)):
    for w in range(len(laser_wavelengths)):
        file_name = file_prefix + "DSC_" + str(img_start[w] + i).zfill(4) + ".NEF"
        file_meas = FileMeasurement(file_name, w * len(theta) + i)

        # Crop
        file_meas.data = file_meas.data[crop_y0:crop_y1, crop_x0:crop_x1, :]

        if w == 0:
            # Blue laser
            # Find percentile of blue channel
            meas_perc = np.percentile(file_meas.data[:,:,2].flatten(), percentile)
            measurements[0,i] = meas_perc
            
        elif w == 1:
            # Green laser
            # Find percentile of green channel
            meas_perc = np.percentile(file_meas.data[:,:,1].flatten(), percentile)
            measurements[1,i] = meas_perc
            
        else:
            # Red laser
            # Find percentile of red channel
            meas_perc = np.percentile(file_meas.data[:,:,0].flatten(), percentile)
            measurements[2,i] = meas_perc

  0%|          | 0/37 [00:00<?, ?it/s]

In [4]:
# Solve for gamma (retardance angle) for each wavelength

x = np.cos(np.deg2rad(2 * theta))
A = np.vstack([x, np.ones(x.shape[0])]).T

cos_gamma = np.zeros((len(laser_wavelengths,)))

for w in range(len(laser_wavelengths)):
    # Solve Eq. 16 by least squares
    m, n = np.linalg.lstsq(A, measurements[w,:], rcond=None)[0]

    # Solve Eq. 20
    cos_gamma[w] = m / n

# m / n should be the same for all color channels, take the average
# cos_gamma = np.average(cos_gamma, axis = 1)

# Clamp to [-1, 1]
cos_gamma = np.clip(cos_gamma, -1.0, 1.0)

# Find smallest possible gamma
gamma_min = np.arccos(cos_gamma)

# Resolve ambiguity in gamma (see Eqs. 21-24)
wav_blue_over_wav_j = laser_wavelengths[0] / laser_wavelengths
print("wav_blue_over_wav_j: ", wav_blue_over_wav_j)
print()

# Solve for N and +/- per wavelength
# Choose option that gives wav_blue_over_wav_j = gamma_j_over_gamma_blue per wavelength
options_str = ['gamma_min', 
               '2 pi - gamma_min',
               '2 pi + gamma_min',
               '4 pi - gamma_min',
               '4 pi + gamma_min'] # etc.

options = [gamma_min, 
           2. * np.pi - gamma_min,
           2. * np.pi + gamma_min,
           4. * np.pi - gamma_min,
           4. * np.pi + gamma_min] # etc.

for i, gamma_blue in enumerate(options):
    print()
    print('Possible solutions if gamma_blue = ', options_str[i])
    
    for j, gamma_j in enumerate(options):
        # Skip all options where gamma_j > gamma_blue (not a possible solution)
        if j > i:
            continue
            
        gj_gb = gamma_j / gamma_blue[0]
        
        # Replace all options where gamma_j_over_gamma_blue > 1 with 0. (not a possible solution)
        # Because we can assume gamma is inversely proportional to wavelength (by Eq. 1)
        gj_gb = np.where(gj_gb > 1.0, 0., gj_gb)
        
        # gamma_blue must equal gamma_j if j = blue
        if i != j:
            gj_gb[0] = 0.
        
        print('gamma_j = ', options_str[j])
        print(gj_gb)
        
# Choose best options, verify that gamma_blue > gamma_green > gamma_red
blue_opt  = 3
green_opt = 2
red_opt   = 2

gamma = np.zeros_like(gamma_min)
gamma[0] = options[blue_opt][0] # blue
gamma[1] = options[green_opt][1] # green
gamma[2] = options[red_opt][2] # red

print()
print("Final gamma: ", gamma)

wav_blue_over_wav_j:  [1.         0.7612782  0.62307692]


Possible solutions if gamma_blue =  gamma_min
gamma_j =  gamma_min
[1.         0.         0.58079826]

Possible solutions if gamma_blue =  2 pi - gamma_min
gamma_j =  gamma_min
[0.         0.46848286 0.11282179]
gamma_j =  2 pi - gamma_min
[1.         0.72577011 0.        ]

Possible solutions if gamma_blue =  2 pi + gamma_min
gamma_j =  gamma_min
[0.         0.33740069 0.08125409]
gamma_j =  2 pi - gamma_min
[0.        0.5226986 0.7788452]
gamma_j =  2 pi + gamma_min
[1.         0.         0.94135338]

Possible solutions if gamma_blue =  4 pi - gamma_min
gamma_j =  gamma_min
[0.         0.21350449 0.05141695]
gamma_j =  2 pi - gamma_min
[0.         0.33075955 0.49284709]
gamma_j =  2 pi + gamma_min
[0.         0.75776852 0.59568098]
gamma_j =  4 pi - gamma_min
[1.         0.87502358 0.        ]

Possible solutions if gamma_blue =  4 pi + gamma_min
gamma_j =  gamma_min
[0.         0.18138854 0.04368266]
gamma_j =  2 pi - gamma_

In [5]:
# Solve for birefringence

inv_wav = 1.0 / laser_wavelengths
A = np.vstack((inv_wav, np.zeros(inv_wav.shape))).T

# Solve Eq. 1 by least squares
bir_thk_2pi, _ = np.linalg.lstsq(A, gamma, rcond=None)[0]

bir_thk = bir_thk_2pi / (2. * np.pi)
birefringence_meas = bir_thk / thickness

print(birefringence_meas)

# Calculate birefringence separately per wavelength instead
birefringence_meas2 = gamma * laser_wavelengths / (2. * np.pi * thickness)
print(birefringence_meas2)

0.009355584390565443
[0.0094504  0.00940683 0.00903487]


In [6]:
print(np.rad2deg(gamma))

# Double check gammas with measured birefringence
gamma_exp = (2. * np.pi) * birefringence_meas * thickness / laser_wavelengths
print(np.rad2deg(gamma_exp))

[661.44367148 501.22119217 394.00941362]
[654.80774659 498.49085972 407.99559595]
